In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24262822
paper_name = 'mattiazziusaj_petrovic_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/MattiazziUsaj_Chemosphere_2014.csv', sep=',')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4308 x 11


In [7]:
original_data.head()

,Plate,Row,Column,ORF,Name,Thiamethoxam,Actara,Acetamiprid,Mospilan,DMSO+Pyrrolidone,Confidor
0,1,2,2,YBR030W,NaN,0.214,-0.010,-0.024,-0.282,-0.025,-0.276
1,1,2,3,YBR138C,HDR1,0.034,0.414,-0.585,-0.095,0.225,-0.227
2,1,2,4,YBR028C,NaN,0.346,0.021,0.216,-1.066,0.025,-0.889
3,1,2,5,YBR137W,NaN,0.314,0.183,0.135,-1.154,0.216,0.317
4,1,2,6,YBR027C,NaN,-0.010,-0.189,-0.141,-0.916,0.093,-0.825


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Plate, Row, Column, ORF, Name, Thiamethoxam, Actara, Acetamiprid, Mospilan, DMSO+Pyrrolidone, Confidor ]
Index: []


In [12]:
original_data.set_index('ORF',inplace=True)
original_data.index.name='orf'

In [13]:
original_data = original_data[['Thiamethoxam','Actara','Acetamiprid','Mospilan','DMSO+Pyrrolidone','Confidor ']].copy()

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(4267, 6)

# Prepare the final dataset

In [16]:
data = original_data.copy()

In [17]:
dataset_ids = [16502,16506,16503,16505,16504,16507]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,16502,16506,16503,16505,16504,16507
data_type,value,value,value,value,value,value
orf,,,,,,
YAL002W,0.071,-0.614,-0.572,-3.279,0.037,-0.774
YAL004W,0.255,0.437,-0.073,-0.284,0.046,0.200
YAL005C,-0.113,0.129,0.125,-0.073,-0.139,-0.105
YAL007C,0.433,0.325,0.124,-0.210,-0.207,0.495
YAL008W,-0.273,0.188,-0.203,-0.711,0.076,-0.054


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16502,16506,16503,16505,16504,16507
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
2,YAL002W,0.071,-0.614,-0.572,-3.279,0.037,-0.774
1863,YAL004W,0.255,0.437,-0.073,-0.284,0.046,0.200
4,YAL005C,-0.113,0.129,0.125,-0.073,-0.139,-0.105
5,YAL007C,0.433,0.325,0.124,-0.210,-0.207,0.495
6,YAL008W,-0.273,0.188,-0.203,-0.711,0.076,-0.054


# Normalize

In [22]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [23]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [24]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16502,16506,16503,16505,16504,16507,16502,16506,16503,16505,16504,16507
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
2,YAL002W,0.071,-0.614,-0.572,-3.279,0.037,-0.774,-0.138219,-0.736667,-0.775827,-3.557405,-0.470235,-1.126143
1863,YAL004W,0.255,0.437,-0.073,-0.284,0.046,0.200,0.390174,0.457529,-0.106413,-0.229629,-0.449897,0.353439
4,YAL005C,-0.113,0.129,0.125,-0.073,-0.139,-0.105,-0.666611,0.107565,0.159206,0.004815,-0.867952,-0.109880
5,YAL007C,0.433,0.325,0.124,-0.210,-0.207,0.495,0.901336,0.330269,0.157865,-0.147407,-1.021616,0.801567
6,YAL008W,-0.273,0.188,-0.203,-0.711,0.076,-0.054,-1.126082,0.174603,-0.280809,-0.704074,-0.382104,-0.032407


# Print out

In [25]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [26]:
from IO.save_data_to_db3 import *

In [27]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 24262822...
Inserting the new data...


100%|██████████| 6/6 [00:40<00:00,  6.73s/it]

Updating the data_modified_on field...
